# ALMA-IMF+SPICY: SED Fitting

Workspace for importing the fits files generated through the SED Table Prep workspace, running the SED fitting, and saving the results to an astropy table.

In [1]:
from table_loading import *

In [2]:
# get rid of ALMA points for comparison?
almaoverride = False

# select ALMA-IMF field of interest
fields_to_fit = None

In [3]:
# utility function to read object sizes in more convenient measurements
def bytesto(bytes, to, bsize=1024):
    """convert bytes to megabytes, etc.
       sample code:
           print('mb= ' + str(bytesto(314575262000000, 'm')))
       sample output: 
           mb= 300002347.946
    """

    a = {'k' : 1, 'm': 2, 'g' : 3, 't' : 4, 'p' : 5, 'e' : 6 }
    r = float(bytes)
    for i in range(a[to]):
        r = r / bsize

    return(r)

In [ ]:
if fields_to_fit == None:
    # Call 'None' to fit all fields in the sample (will take a while!)
    fields_to_fit = ['G10','G12','W43MM1','W43MM2','W43MM3','W51-E','W51IRS2',
                     'G008','G327','G328','G333','G337','G338','G351','G353']

fit_results_tbl = Table(data=np.zeros(0, dtype=[('SPICY', 'int64'), ('geometry', 'str'),
                                            ('source.flux', 'str'),('source.error', 'str'),
                                            ('source.valid', 'str'),('MODEL_NAME', 'str'),
                                            ('chi2', 'float64'),('av', 'float64'),('sc', 'float64')]))

if almaoverride:
    fit_results_tbl.write(f'/blue/adamginsburg/adamginsburg/SPICY_ALMAIMF/BriceTingle/fit_results_NoALMA', format='fits', overwrite=True)
else:
    fit_results_tbl.write(f'/blue/adamginsburg/adamginsburg/SPICY_ALMAIMF/BriceTingle/fit_results', format='fits', overwrite=True)
del(fit_results_tbl)

for fieldid in tqdm(fields_to_fit):
    print(fieldid)
    region_fitinfo = {}

    # load table from Table Prep notebook
    tbl = Table.read(f'/blue/adamginsburg/adamginsburg/SPICY_ALMAIMF/BriceTingle/Region_tables/Unfitted/{fieldid}', format='fits') # !filepath!
    print("Sources: ",len(tbl))

    if almaoverride:
        for x in ['ALMA-IMF_3mm_flux','ALMA-IMF_3mm_eflux','ALMA-IMF_1mm_flux','ALMA-IMF_1mm_eflux']:
            tbl[x] = 0
            tbl[x] = np.nan
            tbl[x] = np.ma.masked

    # define constant sizes, based on literature values
    apertures = [2.4, 2.4, 2.4, 2.4, 6, 10, 13.5, 23, 30, 41, 3, 3]*u.arcsec
    
    # determine filters and add aperture sizes based on whether we're using UKDISS or VIRAC data
    ukidss_fields = ['G10','G12','W43MM1','W43MM2','W43MM3','W51-E','W51IRS2']
    virac_fields = ['G008','G327','G328','G333','G337','G338','G351','G353']
    
    if any(x in virac_fields for x in tbl['ALMAIMF_FIELDID']):
        print("Grabbing VIRAC filters")
        sed_filters, wavelength_dict, filternames, zpts = get_filters("south")
        apertures = apertures.insert(0,[1.415, 1.415, 1.415, 1.415, 1.415]*u.arcsec)
        hemisphere = "south"
    elif any(x in ukidss_fields for x in tbl['ALMAIMF_FIELDID']):
        print("Grabbing UKIDSS filters")
        sed_filters, wavelength_dict, filternames, zpts = get_filters("north")
        apertures = apertures.insert(0,[2, 2, 2]*u.arcsec)
        hemisphere = "north"
    print("Filters:"+str(filternames))
    print("Apertures:"+str(apertures))

    # read distance from table (in kpc)
    regiondistance = np.nanmax(tbl['Distance'])
    print("Region literature distance (kpc): "+str(regiondistance))
    
    # infer allowable distance range (in kpc)
    distance_range=[regiondistance-0.2, regiondistance+0.2]
    print("Distance range (kpc):"+str(distance_range))
    
    # determine appropriate extinction, using 2*region distance as a lower bound
    av_range=[regiondistance*2,60]
    print("Extinction range (kpc): "+str(av_range))
    
    # make extinction law
    extinction = make_extinction()
    
    # run fitting on each source in region, store data to an array
    for rownum, row in enumerate(tbl):
        fits = full_source_fit(tbl, rownum, filternames, apertures, 
                               '/blue/adamginsburg/adamginsburg/SPICY_ALMAIMF/BriceTingle/robitaille_models', 
                               extinction, distance_range*u.kpc, av_range)
        
        # append fit data to data array
        region_fitinfo[tbl['SPICY'][rownum]] = fits
        
        #clean variables for memory
        del(fits)
    
    new_tbl = construct_fitinfo_tbl(region_fitinfo)
    if almaoverride:
        prev_tbl = Table.read('/blue/adamginsburg/adamginsburg/SPICY_ALMAIMF/BriceTingle/fit_results_NoALMA')
    else:
        prev_tbl = Table.read('/blue/adamginsburg/adamginsburg/SPICY_ALMAIMF/BriceTingle/fit_results') 
    fit_results_tbl = vstack([prev_tbl, new_tbl])
    print(bytesto(np.sum([np.sum([sys.getsizeof(fit_results_tbl[colname][n]) for n in np.arange(len(fit_results_tbl))]) for colname in fit_results_tbl.keys()]),'g'))

    del(new_tbl)
    del(prev_tbl)
    del(region_fitinfo)

    if almaoverride:
        fit_results_tbl.write(f'/blue/adamginsburg/adamginsburg/SPICY_ALMAIMF/BriceTingle/fit_results_NoALMA', format='fits', overwrite=True)
    else:
        fit_results_tbl.write(f'/blue/adamginsburg/adamginsburg/SPICY_ALMAIMF/BriceTingle/fit_results', format='fits', overwrite=True)
    del(fit_results_tbl)